In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import random
import time
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
def PROTOTYPE__xyCreateAxisColumn_polars__(self, 
                                           df, 
                                           field, 
                                           is_scalar, 
                                           new_axis_field,
                                           order          = None,   # Order of the values on the axis
                                           fill_transform = True,   # Fill in missing transform values
                                           timestamp_min  = None,   # Minimum timestamp field
                                           timestamp_max  = None,   # Maximum timestamp field
                                           _min           = None,   # Minimum for scalar axis
                                           _max           = None):  # Maximum for scalar axis
    if type(field) != list:
        field = [field]
    is_time = False    
    field_countable = self.fieldIsArithmetic(df, field[0])
    f0 = field[0]
    transFunc = None
    # Numeric scaling // DONE!
    if field_countable and is_scalar and len(field) == 1:
        my_min = df[f0].min() if _min is None else _min
        my_max = df[f0].max() if _max is None else _max
        if my_min == my_max:
            my_min -= 0.5
            my_max += 0.5
        df = df.with_columns(((pl.col(f0)-my_min)/(my_max-my_min)).alias(new_axis_field))
        label_min = str(my_min)
        label_max = str(my_max)
        transFunc = lambda x: ((x - my_min)/(my_max - my_min))
    # Timestamp scaling // DONE!
    elif len(field) == 1 and df[field[0]].dtype == pl.Datetime:
        # Use dataframe for min... or the parameter version if set
        my_min = df[f0].min() if timestamp_min is None else timestamp_min
        my_max = df[f0].max() if timestamp_max is None else timestamp_max
        if my_min == my_max:
            my_max += timedelta(seconds=1)
        df = df.with_columns(((pl.col(f0)-my_min)/(my_max-my_min)).alias(new_axis_field))
        label_min = timestamp_min
        label_max = timestamp_max
        is_time = True
        transFunc = lambda x: ((x - my_min)/(my_max - my_min))    
    # Equal scaling
    else:
        # This fills in the natural ordering of the data if the fill_transform is enabled (it's true by default)
        # ... unclear what should be done if this is multifield and one or more transforms exists
        if fill_transform and order is None and len(field) == 1 and self.isTField(f0):
            order = self.transformNaturalOrder(df, f0)
            order_filled_by_transform = True
        else:
            if fill_transform and order is None and len(field) > 1:
                for _field in field:
                    if self.isTField(_field):
                        raise Exception('xy - fill_transform is specified but there are multiple fields with at least one transform... create your own order...')
            order_filled_by_transform = False

        # Determine all the possibilities in the dataframe
        if len(field) == 1:
            all_combos = sorted(list(set(df[field[0]])))
        else:
            df         = df.sort(field)
            group_by   = df.group_by(field, maintain_order=True)
            all_combos    = []
            for k, k_df in group_by:
                all_combos.append(k)

        # Determine the order & create the dictionary
        if order is None:
            order = all_combos
            # Create the dictionary
            _order_len_ = (len(order)-1) if len(order) > 1 else 1
            _dict_, i = {}, 0
            for x in order:
                _dict_[x] = i/_order_len_
                i += 1
        else:
            gb_set, order_set = set(all_combos), set(order)
            order_is_complete = (len(gb_set) == len(order_set)) and (len(gb_set & order_set) == len(order_set))
            if order_is_complete:
                # Create the dictionary
                _order_len_ = (len(order)-1) if len(order) > 1 else 1
                _dict_, i = {}, 0
                for x in order:
                    _dict_[x] = i/_order_len_
                    i += 1
            else:
                order.append('ee') # last order is the 'everything else' category...
                # Create the dictionary
                _order_len_ = (len(order)-1) if len(order) > 1 else 1
                _dict_, i = {}, 0
                for x in order:
                    _dict_[x] = i/_order_len_
                    i += 1
                for x in (gb_set - order_set):
                    _dict_[x] = 1.0

        # Create the new column from the dictionary
        if len(field) == 1:
            df = df.with_columns(pl.col(field[0]).map_dict(_dict_).alias(new_axis_field))
        else:
            def myMapRows(k):
                return _dict_[k]
            axis_series = df.drop(set(df.columns) - set(field)).select(field).map_rows(myMapRows)['map']
            df = df.with_columns(pl.Series(new_axis_field, axis_series))

        # Compute the min and max labels
        def concatAsStrs(x):
            if type(x) == list:
                s = str(x[0])
                for i in range(1,len(x)):
                    s += '|' + str(x[i])
            else:
                return str(x)
        label_min, label_max = concatAsStrs(order[0]), concatAsStrs(order[-1])

    return df, is_time, label_min, label_max, transFunc, order

In [ ]:
df = pd.DataFrame({'x':[0, 10, 9, 20, 2]})
df_pl = pl.DataFrame(df)
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, 'x', True,  'xaxis')
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, 'x', True,  'xaxis2',  _min=5, _max=10)
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, 'x', False, 'xaxis3')
df

In [ ]:
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, 'x', True,  'xaxis')
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, 'x', True,  'xaxis2',  _min=5, _max=10)
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, 'x', False, 'xaxis3')
df_pl

In [ ]:
df = pd.DataFrame({'ts':['2003-01-01', '2001-10-02', '2010-04-30', '2003-11-30', '2008-03-15']})
df = rt.columnsAreTimestamps(df, 'ts')
df_pl = pl.DataFrame(df)
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, 'ts', False,  'tsaxis')
df

In [ ]:
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, 'ts', True,  'tsaxis')
df_pl

In [ ]:
df    = pd.DataFrame({'field':['a', 'b', 'c', 'a', 'a', 'z', 'b', 'e'],
                      'suba': ['i', 'i', 'j', 'j', 'j', 'i', 'i', 'j'],
                      'sub':  [1,   1,   1,   2,   1,   10,  0,   10]})
df_pl = pl.DataFrame(df)
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, 'field',         False,  'axis')
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, ['field','sub'], False,  'axis2')
df, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df, ['sub','field'], False,  'axis3')
df.sort_values(['field','suba','sub', 'axis', 'axis2', 'axis3'])

In [ ]:
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, 'field',         False,  'axis')
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, ['field','sub'], False,  'axis2')
df_pl, is_time, label_min, label_max, tF, order = rt.xyCreateAxisColumn(df_pl, ['sub','field'], False,  'axis3')
df_pl.sort(['field', 'suba', 'sub', 'axis', 'axis2', 'axis3'])

In [ ]:
df_pl = pl.DataFrame({'x':[1,2,3], 'y':[3,4,5], 'c':['red', 'blue', 'green'], 's':[10, 5, 15]})
rt.xy(df_pl, x_field='x', y_field='y', color_by='c', count_by='s', dot_size='vary')

In [ ]:
df_pl.filter(pl.col('s') >= 10)

In [ ]:
df_pl